In [5]:
import os
import re
import csv
import openpyxl
import pandas as pd
from openpyxl import load_workbook

### Data Profiling
Data profiling was performed on the raw files in the path `/mnt/data/public/customs`. The scope of the data profiling involved:
- Identify number of files in the path
- Identify covered period per file
- Identify the number of sheets per file
- Investigate files that are not in the expected* format
- Convert .xlsx to .csv and save them to path

In [8]:
# Define global variables
root = '/mnt/data/public/customs/' #Path of customs data
current_wd = os.getcwd() #Path of current working directory

# Create directories
try:
    os.mkdir('{}/converted_csv_data'.format(os.path.dirname(current_wd)))
except:
    pass

try:
    os.mkdir('{}/misc'.format(os.path.dirname(current_wd)))
except:
    pass

In [3]:
# Create a list of all .xlsx files in /mnt/data/public/customs/
files = os.listdir(root)
files = [file for file in files if 'xlsx' in file]
len(files)

93

In [5]:
# Comment out incase accidentally re-run
# Initialize variables
dict_files = dict()
df = pd.DataFrame()
# Assign pattern to the following variables for Month and Year
pattern_mnth = (r'(January|February|March|April|May|June|July|August|'
                'September|October|November|December|Mar|Jan|Apr|Jun|Feb)')
pattern_year = r'(2012|2013|2014|2015|2016|2017|2018|2019|2020)'
# Create 
for file in files:
    path = root + file
    match_month = re.findall(pattern_mnth, file)
    match_year = re.findall(pattern_year, file)
    dict_files[files.index(file)] = {'file': file,
                                     'month': ' to '.join(match_month),
                                     'year': ' to '.join(match_year),
                                     'sheets': (load_workbook(path,
                                                              read_only=True)
                                                .sheetnames)}

In [7]:
# Convert dictionary to a dataframe
df.from_dict(dict_files, orient='index')

,file,month,year,sheets
0,DailyFiledEntriesforApril2020.xlsx,April,2020,"[Export Worksheet, SQL]"
1,DailyFiledEntriesforJanuary2020.xlsx,January,2020,[Export Worksheet]
2,DailyFiledEntriesforDecember2019.xlsx,December,2019,[Export Worksheet]
3,DailyFiledEntriesforSeptember2019.xlsx,September,2019,[Export Worksheet]
4,DailyFiledEntriesforAugust2019.xlsx,August,2019,[Export Worksheet]
...,...,...,...,...
88,DailyFiledEntriesforJanuarytoMarch-2013.xlsx,January to March,2013,[Export Worksheet]
89,DailyFiledEntriesforOctobertoDecember-2012.xlsx,October to December,2012,[Export Worksheet]
90,DailyFiledEntriesforJulytoSeptember-2012.xlsx,July to September,2012,[Export Worksheet]
91,DailyFiledEntriesforApriltoJune-2012.xlsx,April to June,2012,[Export Worksheet]


In [8]:
# Identify files for investigation
df = df.from_dict(dict_files, orient='index')
df['number_of_sheets'] = df['sheets'].apply(lambda x: len(x))
df['number_of_sheets'].value_counts()
investigate = df[df['number_of_sheets']>1]

# Investigate those with sheets greater than 1.
investigate

,file,month,year,sheets,number_of_sheets
0,DailyFiledEntriesforApril2020.xlsx,April,2020,"[Export Worksheet, SQL]",2
29,Customs-Entries-for-September-2014.xlsx,September,2014,"[Sheet1, Sheet2, Sheet3]",3
52,Customs-Entries-for-November-2014-Complete.xlsx,November,2014,"[Sheet1, Sheet2, Sheet3]",3
65,Daily-Filed-Entries-for-February-2017.xlsx,February,2017,"[Export Worksheet, Sheet1]",2


In [9]:
# Comment out incase accidentally re-run
# Check contents of the following files
investigate_list = investigate['file'].to_list()

# Initialize variables
dict_investigate = dict()

for file in investigate_list:
    path = root + file
    xls = pd.ExcelFile(path)
    sheets = [(xls.sheet_names[i], xls.parse(i).size)
              for i in range(len(xls.sheet_names)-1)]
    dict_investigate[investigate_list.index(file)] = {'file': file, 
                                                      'sheets': sheets}

In [10]:
# Update list of files to be converted to csv
files_df = df
for file in investigate_list:
    files_df.loc[files_df['file'] == file,['sheets']] =  df['sheets'].map(lambda x: x[0])
files_df['sheets'] = files_df['sheets'].str.join('')
files_df.loc[files_df['year']=='2014 to 2014',['year']] = '2014'
files_df['month_year'] = files_df[['month', 'year']].agg('-'.join, axis=1)
files_df = files_df.drop('number_of_sheets', axis=1)

,file,month,year,sheets,month_year
0,DailyFiledEntriesforApril2020.xlsx,April,2020,Export Worksheet,April-2020
1,DailyFiledEntriesforJanuary2020.xlsx,January,2020,Export Worksheet,January-2020
2,DailyFiledEntriesforDecember2019.xlsx,December,2019,Export Worksheet,December-2019
3,DailyFiledEntriesforSeptember2019.xlsx,September,2019,Export Worksheet,September-2019
4,DailyFiledEntriesforAugust2019.xlsx,August,2019,Export Worksheet,August-2019
...,...,...,...,...,...
88,DailyFiledEntriesforJanuarytoMarch-2013.xlsx,January to March,2013,Export Worksheet,January to March-2013
89,DailyFiledEntriesforOctobertoDecember-2012.xlsx,October to December,2012,Export Worksheet,October to December-2012
90,DailyFiledEntriesforJulytoSeptember-2012.xlsx,July to September,2012,Export Worksheet,July to September-2012
91,DailyFiledEntriesforApriltoJune-2012.xlsx,April to June,2012,Export Worksheet,April to June-2012


In [30]:
# Save list of files to csv to Misc folder
files_df.to_csv('../misc/List_of_Files.csv',index=False)

In [12]:
# Check file count per year
files_df.groupby(['year']).size()

year
2012     4
2013     8
2014    17
2015    12
2016    12
2017    12
2018    12
2019    12
2020     4
dtype: int64

In [13]:
# Check file count per month from year 2014
files_df[files_df['year']=='2014'].groupby('month').size()

month
Apr          1
August       1
December     1
Feb          1
Jan          1
January      5
July         1
June         1
Mar          1
May          1
November     1
October      1
September    1
dtype: int64

In [16]:
# Convert the following files to csv and save it in converted_csv folder
def csv_from_excel():
    for file in files:
        path = root + file
        wb = openpyxl.load_workbook(root+file, read_only=True)
        ws = wb[files_df[files_df['file']==file]['sheets'].values[0]]
        output = ('../converted_csv_data/' +
                  file.replace('.xlsx','') +
                  '_' +
                  'Customs_Data_' + 
                  files_df[files_df['file']==file]['month_year'].values[0] + 
                  '.csv')
        with open(output, 'w') as f:
            c = csv.writer(f)
            for r in ws.rows:
                c.writerow([cell.value for cell in r])

In [18]:
# Uncomment to convert csv_from_excel
# Runs the csv_from_excel function:
csv_from_excel()


In [3]:
csv = os.listdir('../converted_csv_data/')
csv = [file for file in csv if '.csv' in file]
len(csv)

93